# Testing SLSQP
nlopt's SLSQP appears to violate constraints at a "converged" solution. Check this.

## nlopt's SLSQP
### Problem 1

In [1]:
import nlopt
import numpy as np

SOLVER_TOL = 1e-6


def rosenbrock(x, grad):
    if grad.size > 0:
        grad[0] = -400 * x[0] * (x[1] - x[0] ** 2) - 2 * (1 - x[0])
        grad[1] = 200 * (x[1] - x[0] ** 2)
    objf = 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2
    print("objf = ", objf, "at x[0] = ", x[0], " x[1] = ", x[1])
    return objf


def constraint(x, grad):
    if grad.size > 0:
        grad[0] = (2 * x[0]) * 1e10
        grad[1] = (2 * x[1]) * 1e10
    return (x[0] ** 2 + x[1] ** 2 - 4) * 1e10


opt = nlopt.opt(nlopt.LD_SLSQP, 2)
opt.set_min_objective(rosenbrock)
opt.add_equality_constraint(constraint, 1e-8)
opt.set_ftol_rel(SOLVER_TOL)
x_0 = np.array([2.5, 5])
x_opt = opt.optimize(x_0)

minf = opt.last_optimum_value()
print("optimum at ", x_opt[0], x_opt[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

m = 1
cons = np.zeros(m)
print("constraint at minimum = ", constraint(x_opt, np.zeros(2)))

objf =  158.5 at x[0] =  2.5  x[1] =  5.0
objf =  146803505608115.06 at x[0] =  -1100.990000000001  x[1] =  554.0200000000002
objf =  13389984184.061459 at x[0] =  -107.8490000000001  x[1] =  59.90200000000002
objf =  388897.0117672333 at x[0] =  -8.534900000000011  x[1] =  10.490200000000002
objf =  1295.2788393920505 at x[0] =  1.3965099999999988  x[1] =  5.5490200000000005
objf =  14.422721540390352 at x[0] =  2.189481326672275  x[1] =  5.154492530091245
objf =  14.422721540390352 at x[0] =  2.189481326672275  x[1] =  5.154492530091245
objf =  17.743208903111793 at x[0] =  1.759845600424561  x[1] =  2.6827397072793495
objf =  18.12930562844732 at x[0] =  1.4594550668037019  x[1] =  1.706710064497841
objf =  22.524318901599212 at x[0] =  1.107866374815913  x[1] =  1.701843233066463
objf =  0.5491729153694156 at x[0] =  1.2788871605734926  x[1] =  1.7042105712502886
objf =  0.5491729153694156 at x[0] =  1.2788871605734926  x[1] =  1.7042105712502886
objf =  0.06463636845785697 at x[0]

Equality constraint is clearly above the specified tolerance: the solution should not have converged there. What about a really simple problem?

### Problem 2

In [2]:
def objf(x, grad):
    # x^2 + x
    if grad.size > 0:
        grad[0] = 2 * x[0] + 1
    obj = x[0] ** 2 + x[0]
    print("obj = ", obj, "at x[0] = ", x[0])
    return obj


def constraint(x, grad):
    # -(x - 1)
    # < 0 is satisfied
    if grad.size > 0:
        grad[0] = -1
    return -(x[0] - 2)


n = 1
m = 1
opt = nlopt.opt(nlopt.LD_SLSQP, 1)
opt.set_min_objective(objf)
# opt.add_inequality_constraint(constraint, 1e-5)
opt.add_inequality_constraint(constraint, 0.0)
opt.set_ftol_rel(SOLVER_TOL)
x_0 = np.array([2.0])
x_opt = opt.optimize(np.array([1.0]))

minf = opt.last_optimum_value()
print("optimum at ", x_opt[0])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

cons = np.zeros(m)
print("constraint at minimum = ", constraint(x_opt, cons))

obj =  2.0 at x[0] =  1.0
obj =  5.999999999999996 at x[0] =  1.9999999999999991
obj =  3.7499999999999982 at x[0] =  1.4999999999999996
obj =  3.7499999999999982 at x[0] =  1.4999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.999999999999984 at x[0] =  1.999999999999997
obj =  5.999999999999982 at x[0] =  1.9999999999999964
obj =  5.999999999999981 at x[0] =  1.9999999999999962
obj =  5.999999999999981 at x[0] =  1.9999999999999962
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
obj =  5.9999999999999805 at x[0] =  1.999999999999996
optimum at  1.999999999999996
minimum value =  5.9999999999999805
result code =  3
constraint

With zero-tolerance inequality constraint, constraint is > 0 (violated), very slightly. This isn't right either: equality and inequality constraints violated at the solution.

Try something slightly more complex (2 opt params):

### Problem 3

In [3]:
def objf(x, grad):
    # x1^2 + x2
    if grad.size > 0:
        grad[0] = 2 * x[0]
        grad[1] = 1
    obj = x[0] ** 2 + x[1]
    print("obj = ", obj, "at x = ", x)
    return obj


def constraint(x, grad):
    # -(x - 1)
    # < 0 is satisfied
    if grad.size > 0:
        grad[0] = -1
        grad[1] = -1
    return -(x[0] + x[1] - 6)


n = 2
m = 1
opt = nlopt.opt(nlopt.LD_SLSQP, n)
opt.set_min_objective(objf)
# opt.add_inequality_constraint(lambda x, grad: constraint(x, grad), 1e-5)
opt.add_inequality_constraint(lambda x, grad: constraint(x, grad), 0.0)
opt.set_ftol_rel(SOLVER_TOL)
x_0 = np.array([2.0, 2.0])
x_opt = opt.optimize(x_0)

minf = opt.last_optimum_value()
print("optimum at ", x_opt[0], x_opt[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

con_grads = np.zeros(n)
print("constraint at minimum = ", constraint(x_opt, con_grads))

obj =  6.0 at x =  [2. 2.]
obj =  6.7500000000000515 at x =  [1.5 4.5]
obj =  5.756614710213614 at x =  [0.41866913 5.58133087]
obj =  5.750000000000009 at x =  [0.5 5.5]
obj =  5.749999999999994 at x =  [0.5 5.5]
optimum at  0.5000000000000002 5.500000000000009
minimum value =  5.750000000000009
result code =  3
constraint at minimum =  -8.881784197001252e-15


Constraint < 0 (satisfied) now. So in 1/3 cases it's satisfied. Constraint tolerance affects solution vector: constraint is always within tolerance in this case.

## Compare with scipy's SLSQP
### Problem 1

In [4]:
from scipy import optimize

SOLVER_TOL = 1e-6
CONSTRAINT_TOL = 1e-5


def rosenbrock(x):
    objf = 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2
    # print("objf = ", objf, "at x[0] = ", x[0], " x[1] = ", x[1])
    return objf


def constraint(x):
    return (x[0] ** 2 + x[1] ** 2 - 4) * 1e10


x_0 = np.array([2.5, 5.0])
n = x_0.shape[0]
constraints = []
# c < 0 (when feasible (standard))
ineq_constraints = optimize.NonlinearConstraint(
    constraint, -CONSTRAINT_TOL, CONSTRAINT_TOL
)
constraints.append(ineq_constraints)

result = optimize.minimize(
    rosenbrock,
    x_0,
    method="SLSQP",
    jac=None,
    # bounds=bounds,
    constraints=constraints,
    tol=SOLVER_TOL,
    options={"disp": True},
)

# Recalculate constraints at optimium x
x_opt = result.x
opt_val = result.fun
print(f"{opt_val = }")
print(f"{x_opt = }")
print("Constraint = ", constraint(x_opt))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.06225348172983235
            Iterations: 12
            Function evaluations: 45
            Gradient evaluations: 12
opt_val = 0.06225348172983235
x_opt = array([1.24939191, 1.56173617])
Constraint =  -8.881784197001252e-06


Roughly same point as nlopt's problem 1, but constraint is feasible. Reacts to tolerance, but always within it.


### Problem 2

In [5]:
EQ_CONSTRAINT_TOL = 1e-8


def obj_func(x):
    # x^2 + x
    obj = x[0] ** 2 + x[0]
    print("obj = ", obj, "at x[0] = ", x[0])
    return obj


def constraint_ineq_vec(x):
    # -(x - 1)
    # < 0 is satisfied
    return -(x[0] - 2)


x_0 = np.array([2.0])
n = x_0.shape[0]

constraints = []
# c < 0 (when feasible (standard))
ineq_constraints = optimize.NonlinearConstraint(constraint_ineq_vec, -np.inf, 0.0)
constraints.append(ineq_constraints)

result = optimize.minimize(
    obj_func,
    x_0,
    method="SLSQP",
    jac=None,
    # bounds=bounds,
    constraints=constraints,
    tol=SOLVER_TOL,
    options={"disp": True},
)

# Recalculate constraints at optimium x
x_opt = result.x
opt_val = result.fun
print(f"{opt_val = }")
print(f"{x_opt = }")
print("Constraint = ", constraint_ineq_vec(x_opt))

obj =  6.0 at x[0] =  2.0
obj =  6.000000074505806 at x[0] =  2.000000014901161
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.0
            Iterations: 1
            Function evaluations: 2
            Gradient evaluations: 1
opt_val = 6.0
x_opt = array([2.])
Constraint =  -0.0


In constrast to nlopt's SLSQP, this solution is feasible.

### Problem 3

In [6]:
def objf(x):
    # x1^2 + x2
    obj = x[0] ** 2 + x[1]
    # print("obj = ", obj, "at x = ", x)
    return obj


def constraint(x):
    # -(x - 1)
    # < 0 is satisfied
    return -(x[0] + x[1] - 6)


x_0 = np.array([2.0, 2.0])
n = x_0.shape[0]

constraints = []
# c < 0 (when feasible (standard))
ineq_constraints = optimize.NonlinearConstraint(constraint, -np.inf, 0.0)
constraints.append(ineq_constraints)

result = optimize.minimize(
    objf,
    x_0,
    method="SLSQP",
    jac=None,
    # bounds=bounds,
    constraints=constraints,
    tol=SOLVER_TOL,
    options={"disp": True},
)

# Recalculate constraints at optimium x
x_opt = result.x
opt_val = result.fun
print(f"{opt_val = }")
print(f"{x_opt = }")
print("Constraint = ", constraint(x_opt))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.750000000000023
            Iterations: 4
            Function evaluations: 12
            Gradient evaluations: 4
opt_val = 5.750000000000023
x_opt = array([0.49999997, 5.50000003])
Constraint =  -2.220446049250313e-14


Problem 3 gives same as nlopt: constraint is satisfied.

## Conclusion
For the 3 toy problems, nlopt and scipy's SLSQP implementations give very similar results. However, nlopt allows constraints to be violated in 2/3 cases, whereas scipy always satisfies constraints. Therefore only scipy's SLSQP can be guaranteed to produce feasible solutions.